You are provided with daily historical sales data. The task is to forecast the total amount of products sold in every shop for the test set. Note that the list of shops and products slightly changes every month. Creating a robust model that can handle such situations is part of the challenge.
File descriptions

    sales_train.csv - the training set. Daily historical data from January 2013 to October 2015.
    test.csv - the test set. You need to forecast the sales for these shops and products for November 2015.
    sample_submission.csv - a sample submission file in the correct format.
    items.csv - supplemental information about the items/products.
    item_categories.csv  - supplemental information about the items categories.
    shops.csv- supplemental information about the shops.

Data fields

    ID - an Id that represents a (Shop, Item) tuple within the test set
    shop_id - unique identifier of a shop
    item_id - unique identifier of a product
    item_category_id - unique identifier of item category
    item_cnt_day - number of products sold. You are predicting a monthly amount of this measure
    item_price - current price of an item
    date - date in format dd/mm/yyyy
    date_block_num - a consecutive month number, used for convenience. January 2013 is 0, February 2013 is 1,..., October 2015 is 33
    item_name - name of item
    shop_name - name of shop
    item_category_name - name of item category

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
item_cat = pd.read_csv('../input/competitive-data-science-predict-future-sales/item_categories.csv', dtype={'item_category_name': 'str', 'item_category_id': 'int32'})

In [ ]:
item = pd.read_csv('../input/competitive-data-science-predict-future-sales/items.csv', dtype={'item_name': 'str', 'item_id': 'int32', 
                                                 'item_category_id': 'int32'})

In [ ]:
sale_train = pd.read_csv('../input/competitive-data-science-predict-future-sales/sales_train.csv', dtype={'date': 'str', 'date_block_num': 'int32', 'shop_id': 'int32','item_id': 'int32', 'item_price': 'float32', 'item_cnt_day': 'int32'})

In [ ]:
shop = pd.read_csv('../input/competitive-data-science-predict-future-sales/shops.csv', dtype={'shop_name': 'str', 'shop_id': 'int32'})

In [ ]:
test = pd.read_csv('../input/competitive-data-science-predict-future-sales/test.csv', dtype={'ID': 'int32', 'shop_id': 'int32', 
                                                  'item_id': 'int32'})

In [ ]:
item['item_category_id'].value_counts()

In [ ]:
item.loc[item['item_category_id']==40 , :]

In [ ]:
sale_train

In [ ]:
train = sale_train.join(item, on='item_id', rsuffix='_').join(shop, on='shop_id', rsuffix='_').join(item_cat, on='item_category_id', rsuffix='_').drop(['item_id_', 'shop_id_', 'item_category_id_'], axis=1)

In [ ]:
train.drop(['shop_id', 'item_id'], axis=1, inplace=True)

* We Don't Have Any Null Value

In [ ]:
train.isnull().sum()

### see most 50 item for sale :

In [ ]:
train['item_category_name'].value_counts()[:50].plot(kind='barh', figsize=(10, 10))

In [ ]:
train

In [ ]:
train['month'] = train['date'].transform(lambda x: x.split('.')[1])

In [ ]:
train['day'] = train['date'].transform(lambda x: x.split('.')[0])

In [ ]:
train['year'] = train['date'].transform(lambda x: x.split('.')[2])

In [ ]:
train['year'].value_counts().plot(kind='pie', autopct='%f')

In [ ]:
train['month'].value_counts().plot(kind='pie', autopct='%f', figsize=(10, 10))

In [ ]:
train['shop_name'].value_counts()[:50].plot(kind='barh', figsize=(10, 10))

#### We Use value_counts() and box plot to drop the outlier 

In [ ]:
train['item_cnt_day'].plot(kind='box', figsize=(10,20), grid=True)

In [ ]:
train['item_price'].plot(kind='box', figsize=(10,20), grid=True)

In [ ]:
sns.jointplot(x="item_cnt_day", y="item_price", data=train, height=18)
plt.show()

In [ ]:
train = train.query('item_cnt_day >= 0 and item_cnt_day <= 12 and item_price < 400000')

In [ ]:
train

In [ ]:
train.groupby(['item_category_name', 'month']).size().sort_values(ascending=False)[:20].plot(kind='barh', figsize=(20, 10))

In [ ]:
train.groupby(['item_category_name', 'day']).size().sort_values(ascending=False)[:20].plot(kind='bar', figsize=(20, 10))

In [ ]:
train_time = train.set_index('date')